注意力机制
===

# 1.注意力
我们观察事务时，之所以能够快速判断一种事物(允许判断是错误的)，是因为我们大脑能够很快吧注意力放在事物最具有辨识度的部分从而做出判断，而并非是从头到尾的观察一遍事物后，才能有判断结果。于是就产生了注意力机制

# 2.注意力计算规则
它需要三个指定的输入$Q(query),K(key),V(value)$，然后通过计算公式得到注意力的结果，这个结果代表query在key和value作用下的注意力表示，当输入的$Q=K=V$时，称作自注意力计算规则

1. 将$Q,K$进行纵轴拼接，做一次线性变化，在使用softmax处理获得结果最后与$V$做张量乘法
$$Attention(Q,K,V)=Softmax[Linear(Q, K)] \bullet V$$

2. 将$Q,K$进行纵轴拼接，做一次线性变化后再使用tanh函数激活，然后在进行内部求和，最后使用softmax处理获得结果在与$V$做张量乘法
$$Attention(Q,K,V)=Softmax[sum(tanh(Linear(Q, K)))] \bullet V$$

3. 将$Q$与$K^T$作点积运算，然后除以一个缩放系数，在使用softmax处理获得结果最后与$V$作张量乘法
$$Attention(Q,K,V)=softmax(\frac{Q \bullet K^T}{\sqrt{d_k}}) \bullet V$$

当注意力权重矩阵和$V$都是三维张量且第一为代表为batch条数时，则做BMM运算。BMM是一种特殊的张量乘法运算.如果参数1形式是$b \times n \times m$，参数2形状是$b \times m \times p$，则输出为$b \times n \times p$

In [ ]:
import torch
input = torch.randn(10, 3, 4)
mat2 = torch.randn(10, 4, 5)
res = torch.bmm(input, mat2)
res.size()

# 3.注意力机制

## 3.1.注意力机制
注意力机制是注意力计算规则能够应用的深度学习网络的载体，同时包括一些必要的全连接层以及相关张量处理，使其与应用网络融为一体，使自注意力计算规则的注意力机制成为自注意力机制。在NLP领域中，当前的注意力机制大多数应用于seq2seq架构，即编码器和解码器。

# 3.2.注意力机制的作用
- 在加码器端的注意力机制：能够根据模型目标有效的聚焦编码器的输出结果，当其作为解码器的输入时提升效果，改善以往编码器输出是单一定长张量，无法存储过多信息的情况
- 在编码器端的注意力机制：主要解决表征问题，相当于特征提取过程，得到输入的注意力表示。一般使用自注意力(self-attention)

## 3.3.注意力机制的实现步骤
1. 根据注意力计算规则，对$Q, K, V$进行相应的计算
2. 根据第一步采用的计算方法，如果是拼接方法，则需要将$Q$与第二步的计算结果在进行拼接，如果是转置点积，一般是自注意力，$Q$与$V$相同，则不需要进行与$Q$的拼接
3. 最后为了使整个attention机制按照指定尺寸输出，使用线性层作用在第二步的结果上做一个线性变换，得到最终对$Q$的注意力表示

# 4.注意力机制的代码分析

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Attn(nn.Module):
    def __init__(self, query_size, key_size, value_size1, value_size2, output_size):
        super(Attn, self).__init__()
        self.query_size = query_size
        self.key_size = key_size
        self.value_size1 = value_size1
        self.value_size2 = value_size2
        self.output_size = output_size

        self.attn = nn.Linear(self.query_size + self.key_size, value_size1)
        self.attn_combine = nn.Linear(self.query_size + value_size2, output_size)
    
    def forward(self, Q, K, V):
        